### Group 3 - Task Manager with Priority Queue
---
**Task Description:** <br>
> The Task Manager with Priority Queue is a tool designed to help people stay organized and
productive by managing their tasks efficiently. It allows users to add tasks, set their priority levels
(High, Medium, or Low), and keep track of what needs to be done. Tasks are automatically sorted
by priority, so the most important ones are always at the top.

#### 1. Install and import necessary libraries

In [1]:
!pip install tkcalendar

In [2]:
import tkinter as tk
from tkinter import *
from tkinter import ttk, messagebox
from tkcalendar import *

In [3]:
import json
import os
from datetime import datetime

### Back-end

**Task Class - Encapsulation of Task details**

In [4]:
class Task:
    def __init__(self, task_name, priority, due_date):
        self.id = None
        self.name = task_name
        self.priority = priority
        self.due_date = due_date
        self.status = 'TO DO'
    
    def mark_completed(self):
        self.status = 'COMPLETED'

**TaskManager Class - Manages the collection of tasks (Encapsulation & Abstraction)**

In [5]:
class TaskManager:
    def __init__(self, filename='tasks.json'):
        self.filename = filename
        self.tasks = self.load_tasks()

    def load_tasks(self):
        if not os.path.exists(self.filename):
            return []
        try:
            with open(self.filename, 'r') as f:
                return json.load(f)
        except (json.JSONDecodeError, IOError):
            return []

    def save_tasks(self):
        with open(self.filename, 'w') as f:
            json.dump(self.tasks, f, indent=4)

    def generate_unique_id(self):
        return len(self.tasks) + 1

    def add_task(self, name, priority, due_date):
        if not name or not priority or not due_date:
            return False
        
        task = {
            'id': self.generate_unique_id(),
            'name': name,
            'priority': priority,
            'due_date': due_date,
            'status': 'TO DO'
        }
        self.tasks.append(task)
        self.save_tasks()
        return True

    def get_tasks(self, status=None):
        if status is None:
            return self.tasks
        return [task for task in self.tasks if task['status'] == status]

    def update_task_status(self, task_id, new_status):
        for task in self.tasks:
            if task['id'] == task_id:
                task['status'] = new_status
                self.save_tasks()
                return True
        return False

    def remove_task(self, task_id):
        self.tasks = [task for task in self.tasks if task['id'] != task_id]
        self.save_tasks()

    def search_tasks(self, query):
        return [task for task in self.tasks if query.lower() in task['name'].lower()]

    def export_tasks(self):
        export_filename = 'tasks_export.csv'

        with open(export_filename, 'w') as f:
            f.write("ID,Name,Priority,Due Date,Status\n")

            for task in self.tasks:
                safe_name = task['name'].replace(',', ';')

                line = f"{task['id']},{safe_name},{task['priority']},{task['due_date']},{task['status']}\n"
                f.write(line)

        full_path = os.path.abspath(export_filename)
        messagebox.showinfo("Export Successful", f"Tasks exported to:\n{full_path}")

        return full_path

### Graphical User Interface

**App Class - Main Class that runs the app**

In [6]:
class App(tk.Tk):
    def __init__(self, title, size):
        super().__init__()
        self.title(title)
        self.geometry(f"{size[0]}x{size[1]}")
        self.minsize(size[0], size[1])

        self.task_manager = TaskManager()

        self.sidebar = Sidebar(self, self.task_manager)
        self.main = Main(self, self.task_manager)

        self.update_task_views()

        self.mainloop()

    def update_task_views(self):
        status_map = {
            'ALL': None,
            'TO DO': 'TO DO',
            'IN PROGRESS': 'IN PROGRESS',
            'COMPLETED': 'COMPLETED'
        }
        
        for tab_name, status in status_map.items():
            treeview = getattr(self.main, f'task{list(status_map.keys()).index(tab_name) + 1}')
            
            for item in treeview.get_children():
                treeview.delete(item)
            
            tasks = self.task_manager.get_tasks(status)
            for task in tasks:
                treeview.insert('', 'end', text=task['id'], 
                                values=(task['name'], task['priority'], task['due_date']))

**Sidebar Class - Handles task input and exporting**

In [7]:
class Sidebar(tk.Frame):
    def __init__(self, parent, task_manager):
        super().__init__(parent)
        self.task_manager = task_manager
        self.configure(bg="gray63")
        self.place(x=0, y=0, relwidth=0.28, relheight=1)

        self.create_widgets()
        self.layout_widgets()

    def create_widgets(self):
        # Create the widgets
        self.title = tk.Label(self, text="TASK MANAGER", background="gray63", font=("Helvetica", 12, "bold"))
        self.task_name = tk.Label(self, text="Task name:", background="gray63", font=("Helvetica", 10, "bold"))
        self.name_entry = tk.Entry(self)
        self.priority = tk.Label(self, text="Priority:", background="gray63", font=("Helvetica", 10, "bold"))
        self.priority_combo = ttk.Combobox(self, values=["Low", "Medium", "High"])
        self.due_date = tk.Label(self, text="Due date:", background="gray63", font=("Helvetica", 10, "bold"))
        self.date_entry = DateEntry(self, selectmode="day", date_pattern="mm-dd-yyyy")
        self.add_task = tk.Button(self, text="Add task", background="dodgerblue2", command=self.on_add_task)
        self.export = tk.Button(self, text="Export", background="gray70", command=self.on_export)

        # Create the grid
        self.columnconfigure((0,1), weight=1)
        self.rowconfigure((0,10), weight=1)
        self.rowconfigure(8, weight=25)

    def layout_widgets(self):
        self.title.grid(row=0, column=0, padx=20, pady=12, sticky="w")
        self.task_name.grid(row=1, column=0, padx=20, sticky="w")
        self.name_entry.grid(row=2, column=0, columnspan=2, padx=20, pady=8, sticky="nsew")
        self.priority.grid(row=3, column=0, padx=20, sticky="w")
        self.priority_combo.grid(row=4, column=0, columnspan=2, padx=20, pady=8, sticky="nsew")
        self.due_date.grid(row=5, column=0, padx=20, sticky="w")
        self.date_entry.grid(row=6, column=0, columnspan=2, padx=20, pady=8, sticky="nsew")
        self.add_task.grid(row=7, column=0, columnspan=2, padx=20, pady=8, sticky="ew")
        self.export.grid(row=9, column=0, columnspan=1, padx=20, pady=0, sticky="ew")

    def on_add_task(self):
        name = self.name_entry.get()
        priority = self.priority_combo.get()
        due_date = self.date_entry.get()

        if self.task_manager.add_task(name, priority, due_date):
            self.name_entry.delete(0, tk.END)
            self.priority_combo.set('')
            self.date_entry.set_date(datetime.now())

            self.master.update_task_views()
        else:
            messagebox.showerror("Error", "Please fill in all task details")

    def on_export(self):
        exported_file = self.task_manager.export_tasks()
        messagebox.showinfo("Export Successful", f"Tasks exported to {exported_file}")

**Main Class - Displays tasks in tabs (TO DO, IN PROGRESS, COMPLETED, ALL)**

In [8]:
class Main(ttk.Frame):
    def __init__(self, parent, task_manager):
        super().__init__(parent)
        self.task_manager = task_manager
        self.place(relx=0.28, y=0, relwidth=0.72, relheight=1)

        self.create_widgets()
        self.layout_widgets()

    def create_widgets(self):
        self.search_entry = ttk.Entry(self)
        self.search_button = tk.Button(self, text="Search tasks", background="dodgerblue2", command=self.on_search)

        self.manager = ttk.Notebook(self)
        self.tab1 = tk.Frame(self.manager)
        self.tab2 = tk.Frame(self.manager)
        self.tab3 = tk.Frame(self.manager)
        self.tab4 = tk.Frame(self.manager)

        self.manager.add(self.tab1, text="ALL")
        self.manager.add(self.tab2, text="TO DO")
        self.manager.add(self.tab3, text="IN PROGRESS")
        self.manager.add(self.tab4, text="COMPLETED")

        self.task1 = self.create_treeview(self.tab1)
        self.task2 = self.create_treeview(self.tab2)
        self.task3 = self.create_treeview(self.tab3)
        self.task4 = self.create_treeview(self.tab4)

        self.remove_button = tk.Button(self, text="Remove task", background="gray70", command=self.on_remove_task)
        self.update_button = tk.Button(self, text="Update task", background="dodgerblue2", command=self.on_update_task)

    def create_treeview(self, parent):
        treeview = ttk.Treeview(parent, columns=("Column 1", "Column 2", "Column 3"))
        treeview.heading("#0", text="ID", anchor=tk.W)
        treeview.heading("Column 1", text="Task name", anchor=tk.W)
        treeview.heading("Column 2", text="Priority", anchor=tk.W)
        treeview.heading("Column 3", text="Due date", anchor=tk.W)
        treeview.pack(side="left", fill="both", expand=True)
        return treeview

    def layout_widgets(self):
        self.columnconfigure((0, 1, 2, 3), weight=1)
        self.rowconfigure(0, weight=1)
        self.rowconfigure(1, weight=22)
        self.rowconfigure(2, weight=1)

        self.search_entry.grid(row=0, column=0, columnspan=3, padx=12, pady=12, sticky="ew")
        self.search_button.grid(row=0, column=3, padx=12, pady=12, sticky="ew")

        self.manager.grid(row=1, column=0, columnspan=4, padx=12, sticky="nsew")

        self.remove_button.grid(row=2, column=0, padx=12, pady=12, sticky="ew")
        self.update_button.grid(row=2, column=3, padx=12, pady=12, sticky="ew")

    def on_search(self):
        query = self.search_entry.get()
        results = self.task_manager.search_tasks(query)

        for treeview in [self.task1, self.task2, self.task3, self.task4]:
            for item in treeview.get_children():
                treeview.delete(item)

        for task in results:
            self.task1.insert('', 'end', text=task[0], 
                               values=(task[1], task[2], task[3]))

    def on_remove_task(self):
        current_tab_index = self.manager.index(self.manager.select())
        treeviews = [self.task1, self.task2, self.task3, self.task4]
        current_treeview = treeviews[current_tab_index]

        selected_item = current_treeview.selection()
        if not selected_item:
            messagebox.showerror("Error", "Please select a task to remove")
            return

        task_id = current_treeview.item(selected_item[0])['text']
        
        self.task_manager.remove_task(task_id)
        
        self.master.update_task_views()

    def on_update_task(self):
        current_tab_index = self.manager.index(self.manager.select())
        treeviews = [self.task1, self.task2, self.task3, self.task4]
        current_treeview = treeviews[current_tab_index]

        selected_item = current_treeview.selection()
        if not selected_item:
            messagebox.showerror("Error", "Please select a task to update")
            return

        task_id = current_treeview.item(selected_item[0])['text']
        
        status_cycle = ['TO DO', 'IN PROGRESS', 'COMPLETED']
        current_status = ['ALL', 'TO DO', 'IN PROGRESS', 'COMPLETED'][current_tab_index]
        
        if current_status == 'ALL':
            messagebox.showerror("Error", "Please select a specific status tab to update")
            return
        
        current_status_index = status_cycle.index(current_status)
        next_status = status_cycle[(current_status_index + 1) % len(status_cycle)]
        
        self.task_manager.update_task_status(task_id, next_status)
        
        self.master.update_task_views()

In [10]:
App("Task Manager App", (1136, 644))

<__main__.App object .>